### Import libraries

In [1]:
from PIL import Image
import re
from PIL import ImageDraw
from PIL import ImageFont
import pandas as pd
from os import listdir, mkdir
from os.path import isfile, join, exists

### Get Iot component name

In [3]:
def get_file_name(filename):
    items = re.findall('./[a-zA-Z]*/._(.*).PNG$', filename)
    if len(items) > 0:
        return items[0]
    else:
        items = re.findall('./[a-zA-Z]*/(.*).PNG$', filename)
        if len(items) > 0:
            return items[0]
        else:
            return 'default'


### Read Ecosystem image file

In [4]:
def read_ecosystem_image(filename):    
    ecosystem_img_raw = Image.open(filename)
    ecosystem_name = get_file_name(ecosystem_img_raw.filename)
    ecosystem_img = ecosystem_img_raw.resize(
        (300, 300),
        resample=Image.BICUBIC)
    return (ecosystem_name, ecosystem_img)


### Read IoT component image File

In [5]:
def read_iot_image(filename):
    components_width = 100
    components_height = 100
    iot_img = Image.open(filename)
    iot_img_name = get_file_name(iot_img.filename)
    iot_img_resized = iot_img.resize(
    (components_width, components_height),
    resample=Image.BICUBIC)
    return (iot_img_name, iot_img_resized)


### Add Solution texts

In [6]:
def get_application_text_coordinates(solutions_image, draw, text, font):
    left,top,right,bottom = draw.textbbox((10,260), align="center", text=text, font=font)
    x = int((solutions_image.width - right) / 2)
    y = top
    return x, y

def add_solution_texts(solutions_image, iot_component_name, application_text):
    draw = ImageDraw.Draw(solutions_image)
    font = ImageFont.truetype("arial-font/arial.ttf", 18)
    
    draw.text((10, 5),iot_component_name,(255,255,255), font=font)
    
    x, y = get_application_text_coordinates(solutions_image, draw, application_text, font)
    draw.text((x, y),application_text,(36, 116, 54), font=font)


### Create Solution Image

In [7]:
def create_solution_image(ecosystem_image, sensor_image, microcontroller_image, actuator_image, iot_component_name, application_text):
    solutions_img = Image.new("RGB", (300,300))
    components_width = int(solutions_img.width / 3)
    components_height = int(solutions_img.height / 3)
    solutions_img.paste(ecosystem_image, (0, 0))
    solutions_img.paste(sensor_image, (0, components_height))
    solutions_img.paste(microcontroller_image, (components_width, components_height))
    solutions_img.paste(actuator_image, (components_width * 2, components_height))
    add_solution_texts(solutions_img, iot_component_name, application_text)
    return solutions_img


### Read sensor purpose from excell file

In [8]:
def read_applications_file(path, sheet_name):
    df = pd.read_excel(path, sheet_name=sheet_name)    
    df_shrunk = df.loc[:, ['Sensor', 'Actuator', 'Purpose']]
    df_shrunk['Sensor'] = df_shrunk['Sensor'].apply(lambda x: str(x).strip().lower() if type(x) == str else None)
    df_shrunk['Actuator'] = df_shrunk['Actuator'].apply(lambda x: str(x).strip().lower() if type(x) == str else None)
    df_shrunk['Purpose'] = df_shrunk['Purpose'].apply(lambda x: str(x).strip() if type(x) == str else None)    
    df_shrunk = df_shrunk.dropna(how='any')
    return df_shrunk

### Fetch sensor purpose

In [9]:
def get_purpose(df, sensor, actuator):
    sensor_df = df.query(f"Sensor == '{sensor}'")
    # Get the row index of the record queried
    result_index = list(sensor_df.to_dict()['Purpose'].keys())
    if len(result_index) > 0:
        purpose = sensor_df['Purpose'].get(result_index[0])
        return purpose
    else:
        return ''


### List directory files

In [10]:
def list_files(directory_path):
    onlyfiles = [f for f in listdir(directory_path) if isfile(join(directory_path, f))]
    return (directory_path, onlyfiles)


### Generate Solution images

In [12]:
def prepare_iot_component(components_path, solutions_base_dir, hasDir=True, isEcosystem=False):
    (component_name, component_image) = read_ecosystem_image(components_path) if isEcosystem else read_iot_image(components_path)
    iot_component_dir = join(solutions_base_dir, component_name)
    if not exists(iot_component_dir) and hasDir:
        mkdir(iot_component_dir)
    return (component_name, component_image, iot_component_dir)

def generate_solutions(filenames, solutions_path, purpose_df):
    (sensors_dir, sensors) = filenames['sensors']
    (microcontrollers_dir, microcontrollers) = filenames['microcontrollers']
    (actuators_dir, actuators) = filenames['actuators']
    (ecosystems_dir, ecosystems) = filenames['ecosystems']
    
    for sensor in sensors:
        (sensor_name, sensor_image, sensor_dir) = prepare_iot_component(join(sensors_dir, sensor), solutions_path)
        for microcontroller in microcontrollers:
            (microcontroller_name, microcontroller_image, microcontroller_dir) = prepare_iot_component(join(microcontrollers_dir, microcontroller), sensor_dir)
            for ecosystem in ecosystems:
                (ecosystem_name, ecosystem_image, ecosystem_dir) = prepare_iot_component(join(ecosystems_dir, ecosystem), microcontroller_dir, isEcosystem=True)
                for actuator in actuators:
                    (actuator_name, actuator_image, *others) = prepare_iot_component(join(actuators_dir, actuator), ecosystem_dir, hasDir=False)
                    purpose = get_purpose(purpose_df, sensor_name.lower(), actuator_name.lower())
                    solutions_img = create_solution_image(ecosystem_image, sensor_image, microcontroller_image, actuator_image, sensor_name, purpose)
                    solutions_img.save(f"{ecosystem_dir}/{sensor_name}_{microcontroller_name}_{actuator_name}_{ecosystem_name}.png")


### Main

In [15]:
actuators_path = './actuators'
sensors_path = './sensors'
microcontrollers_path = './microcontrollers'
ecosystems_path = './ecosystems'
purpose_df = read_applications_file('./applications-raw.xlsx', 'Application-Names')

generate_solutions(
    {
        'sensors': list_files(sensors_path),
        'microcontrollers': list_files(microcontrollers_path),
        'actuators': list_files(actuators_path),
        'ecosystems': list_files(ecosystems_path)
    },
    'solutions',
    purpose_df
)
print('Done!')

Done!
